In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from django.db.models import Sum, Max, Subquery, Q, F, Avg, Count, Min, Max, Sum, F, Q
from django.db.models import Value, IntegerField, When, FloatField
from django.db.models.functions import Coalesce, Greatest, Floor
from account.models import User
from booth.models import Booth, Transaction, Participation
from player.models import Player
import random
import pandas as pd


In [8]:
user_df = pd.DataFrame(User.objects.all().values())
user_df['url'] = 'https://lionslifegame.com/player/' + user_df['encrypted_id']
user_df

,id,password,last_login,is_superuser,username,email,is_staff,is_active,date_joined,user_type,first_name,last_name,nick_name,mobile,school,encrypted_id,school_code,instructor_group_id,url
0,1,pbkdf2_sha256$390000$Z23pzUJVWJMhBWBEq3jkJa$uf...,2025-02-10 11:29:13.095470+00:00,True,admin,wanyichun@gmail.com,True,True,2025-02-05 16:57:25.521758+00:00,None,,,None,None,None,5bd7QAuv2Sv3te5JDX5zWMsGoDjEaPfW,None,1.0,https://lionslifegame.com/player/5bd7QAuv2Sv3t...
1,101,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,student_101,,False,True,2025-02-07 12:33:32.654121+00:00,student,Player,1,nan,None,None,fndaJK2zKw1oD2rWgKtlHtSgJLe3MNgZ,None,1.0,https://lionslifegame.com/player/fndaJK2zKw1oD...
2,102,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,student_102,,False,True,2025-02-07 12:33:32.690162+00:00,student,Player,2,nan,None,None,yt9jmrjFAplkowVghfnijiMOnWycs84B,None,1.0,https://lionslifegame.com/player/yt9jmrjFAplko...
3,103,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,student_103,,False,True,2025-02-07 12:33:32.725011+00:00,student,Player,3,nan,None,None,lPrT5MSC5nkUHgLLndSTMdPC8Q3meYgB,None,1.0,https://lionslifegame.com/player/lPrT5MSC5nkUH...
4,104,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,student_104,,False,True,2025-02-07 12:33:32.759145+00:00,student,Player,4,nan,None,None,ETWUxYQKT6BLrfNCMLcpnbJUGXCrqR72,None,1.0,https://lionslifegame.com/player/ETWUxYQKT6BLr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,944,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,Z3,,False,True,2025-02-07 12:33:32.486365+00:00,oc,Z3,Z3,nan,None,None,yuHceqAlAI7bS2t6FHaDU2BSO0NktM5R,None,1.0,https://lionslifegame.com/player/yuHceqAlAI7bS...
842,945,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,Z4,,False,True,2025-02-07 12:33:32.520006+00:00,oc,Z4,Z4,nan,None,None,dYdKQ5dooiWLzglgs8q0zeWsX6yigNM3,None,1.0,https://lionslifegame.com/player/dYdKQ5dooiWLz...
843,946,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,Z5,,False,True,2025-02-07 12:33:32.557265+00:00,oc,Z5,Z5,nan,None,None,8nRk8r9Z5hUz42c66sY8vXzWScS1jX0L,None,1.0,https://lionslifegame.com/player/8nRk8r9Z5hUz4...
844,947,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,Z6,,False,True,2025-02-07 12:33:32.590154+00:00,oc,Z6,Z6,nan,None,None,O353dZ3bEuWTHAnxTgkqzJfAWsrMvFAz,None,1.0,https://lionslifegame.com/player/O353dZ3bEuWTH...


,id,password,last_login,is_superuser,username,email,is_staff,is_active,date_joined,user_type,first_name,last_name,nick_name,mobile,school,encrypted_id,school_code,instructor_group_id,url
1,101,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,student_101,,False,True,2025-02-07 12:33:32.654121+00:00,student,Player,1,nan,None,None,fndaJK2zKw1oD2rWgKtlHtSgJLe3MNgZ,None,1.0,https://lionslifegame.com/player/fndaJK2zKw1oD...
2,102,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,student_102,,False,True,2025-02-07 12:33:32.690162+00:00,student,Player,2,nan,None,None,yt9jmrjFAplkowVghfnijiMOnWycs84B,None,1.0,https://lionslifegame.com/player/yt9jmrjFAplko...
3,103,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,student_103,,False,True,2025-02-07 12:33:32.725011+00:00,student,Player,3,nan,None,None,lPrT5MSC5nkUHgLLndSTMdPC8Q3meYgB,None,1.0,https://lionslifegame.com/player/lPrT5MSC5nkUH...
4,104,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,student_104,,False,True,2025-02-07 12:33:32.759145+00:00,student,Player,4,nan,None,None,ETWUxYQKT6BLrfNCMLcpnbJUGXCrqR72,None,1.0,https://lionslifegame.com/player/ETWUxYQKT6BLr...
5,105,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,student_105,,False,True,2025-02-07 12:33:32.793427+00:00,student,Player,5,nan,None,None,NL5AppOi6nw2I86855eNopBhgKDiaf3o,None,1.0,https://lionslifegame.com/player/NL5AppOi6nw2I...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,944,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,Z3,,False,True,2025-02-07 12:33:32.486365+00:00,oc,Z3,Z3,nan,None,None,yuHceqAlAI7bS2t6FHaDU2BSO0NktM5R,None,1.0,https://lionslifegame.com/player/yuHceqAlAI7bS...
842,945,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,Z4,,False,True,2025-02-07 12:33:32.520006+00:00,oc,Z4,Z4,nan,None,None,dYdKQ5dooiWLzglgs8q0zeWsX6yigNM3,None,1.0,https://lionslifegame.com/player/dYdKQ5dooiWLz...
843,946,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,Z5,,False,True,2025-02-07 12:33:32.557265+00:00,oc,Z5,Z5,nan,None,None,8nRk8r9Z5hUz42c66sY8vXzWScS1jX0L,None,1.0,https://lionslifegame.com/player/8nRk8r9Z5hUz4...
844,947,pbkdf2_sha256$390000$Rd0scfcAkVNSrZXBMvcreU$v+...,NaT,False,Z6,,False,True,2025-02-07 12:33:32.590154+00:00,oc,Z6,Z6,nan,None,None,O353dZ3bEuWTHAnxTgkqzJfAWsrMvFAz,None,1.0,https://lionslifegame.com/player/O353dZ3bEuWTH...


In [14]:

import qrcode
img = qrcode.make(user_df.iloc[0]['url'])
type(img)  # qrcode.image.pil.PilImage
img.save("some_file.png")

In [18]:
for i, r in user_df.iloc[1:].iterrows():
    img = qrcode.make(r['url'])
    type(img)  # qrcode.image.pil.PilImage
    img.save(f"../qrcodes/{r.id}.png")